In [ ]:
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install statsmodels
!pip install seaborn
!pip install scipy.stats
!pip install scikit_posthocs

In [ ]:
'''PART 1: LINE PLOT GENERATION FOR ABSORBED PAR ON EACH RANK.'''

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import sem, t

# Load datasets for high and low densities
data_high = pd.read_csv('combined_files/combined_high_ranks_cleaned.csv')
data_low = pd.read_csv('combined_files/combined_low_ranks_cleaned.csv')

# Light input from the lamps
lamp_input = 273.052  # micromols m-2 s-1

# Function to calculate mean, std dev, confidence intervals, and percentage absorption
def calculate_statistics(data, lamp_input, confidence=0.95):
    grouped = data.groupby(['architecture', 'rank'])['absorbedPAR_umol_m2_s1']
    mean_values = grouped.mean().reset_index(name='mean')
    mean_values['std_dev'] = grouped.std().reset_index(name='std_dev')['std_dev']
    mean_values['n'] = grouped.count().reset_index(name='count')['count']
    
    # Calculate the t-value for 95% confidence interval
    mean_values['t_value'] = mean_values['n'].apply(lambda x: t.ppf((1 + confidence) / 2., x - 1))
    
    # Calculate the margin of error
    mean_values['margin_error'] = mean_values['t_value'] * (mean_values['std_dev'] / np.sqrt(mean_values['n']))
    
    # Calculate lower and upper confidence bounds
    mean_values['ci_lower'] = mean_values['mean'] - mean_values['margin_error']
    mean_values['ci_upper'] = mean_values['mean'] + mean_values['margin_error']
    
    # Calculate percentage absorption
    mean_values['percent_absorption'] = (mean_values['mean'] / lamp_input) * 100
    
    return mean_values

# Calculate statistics for both high-density and low-density datasets
mean_values_high = calculate_statistics(data_high, lamp_input)
mean_values_low = calculate_statistics(data_low, lamp_input)

# Plotting function to avoid repetition
def plot_absorbedPAR(mean_values, y_column, y_label, density_label, output_file):
    plt.figure(figsize=(10, 6))
    
    # Plotting lineplot for each architecture type within the current density
    for architecture in mean_values['architecture'].unique():
        architecture_data = mean_values[mean_values['architecture'] == architecture]
        sns.lineplot(x='rank', y=y_column, data=architecture_data, label=architecture, marker='o')
        
        # Adding the confidence interval as a shaded area, if applicable
        if 'ci_lower' in mean_values.columns and y_column == 'mean':
            plt.fill_between(architecture_data['rank'], architecture_data['ci_lower'], architecture_data['ci_upper'], alpha=0.3)

    # Automatically determine the y-axis label based on the output file name
    if 'percent' in output_file.lower():
        y_label = 'Absorbed PAR (%)'
        y_lim = (0, 100)
    else:
        y_label = 'Absorbed PAR umol m-2 s-1'
        y_lim = (0, 300)
    
    # Set plot labels and limits
    plt.xlabel('Leaf Rank')
    plt.ylabel(y_label)
    plt.ylim(y_lim)
    plt.xticks(range(1, 11))
    plt.legend(title='Architecture Type')
    plt.savefig(output_file)
    plt.close()

# Plot for absorbed PAR for high-density data
plot_absorbedPAR(mean_values_high, 'mean', 'Absorbed PAR (umol m-2 s-1)', 'High', 'plots/high_absorbedPAR_ranks.png')

# Plot for percentage absorption for high-density data
plot_absorbedPAR(mean_values_high, 'percent_absorption', 'Absorption (%)', 'High', 'plots/percent_absorption_for_high_density.png')

# Plot for absorbed PAR for low-density data
plot_absorbedPAR(mean_values_low, 'mean', 'Absorbed PAR (umol m-2 s-1)', 'Low', 'plots/low_absorbedPAR_ranks.png')

# Plot for percentage absorption for low-density data
plot_absorbedPAR(mean_values_low, 'percent_absorption', 'Absorption (%)', 'Low', 'plots/percent_absorption_for_low_density.png')

print("Plots generated successfully for high and low densities, including percentage absorption.")

In [11]:
"""PART 2: BAR PLOTS AND BOX PLOTS FOR ABSORBED PAR PER PLANT AND CROP."""

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

# Load datasets for high and low densities
data_high_totalPAR_plants = pd.read_csv('combined_files/combined_total_absorbedPAR_high_cleaned.csv')
data_low_totalPAR_plants = pd.read_csv('combined_files/combined_total_absorbedPAR_low_cleaned.csv')
data_high_crop = pd.read_csv('combined_files/combined_high_sensors_cleaned.csv')
data_low_crop = pd.read_csv('combined_files/combined_low_sensors_cleaned.csv')

# Dictionaries for significance letters
new_names = {        
    'architecture_A': 'A-Gradient45-90',
    'architecture_B': 'B-Gradient90-45',
    'architecture_C': 'C-Uniform90',
    'architecture_D': 'D-Uniform70',
    'architecture_E': 'E-Uniform50',
    'architecture_F': 'F-Uniform30'
}

significance_letters_high_plant = {
    new_names['architecture_A']: 'c',
    new_names['architecture_B']: 'b',
    new_names['architecture_C']: 'a',
    new_names['architecture_D']: 'b',
    new_names['architecture_E']: 'd',
    new_names['architecture_F']: 'e',
    'control': 'd'
}

significance_letters_low_plant = {
    new_names['architecture_A']: 'd',
    new_names['architecture_B']: 'b',
    new_names['architecture_C']: 'a',
    new_names['architecture_D']: 'ab',
    new_names['architecture_E']: 'e',
    new_names['architecture_F']: 'f',
    'control': 'c'
}

significance_letters_high_sensors_crop = {
    new_names['architecture_A']: 'b',
    new_names['architecture_B']: 'd',
    new_names['architecture_C']: 'a',
    new_names['architecture_D']: 'b',
    new_names['architecture_E']: 'e',
    new_names['architecture_F']: 'f',
    'control': 'c'
}

significance_letters_low_sensors_crop = {
    new_names['architecture_A']: 'b',
    new_names['architecture_B']: 'c',
    new_names['architecture_C']: 'a',
    new_names['architecture_D']: 'b',
    new_names['architecture_E']: 'd',
    new_names['architecture_F']: 'e',
    'control': 'b'
}

significance_letters_high_sensors = {
    new_names['architecture_A']: 'e',
    new_names['architecture_B']: 'c',
    new_names['architecture_C']: 'f',
    new_names['architecture_D']: 'e',
    new_names['architecture_E']: 'b',
    new_names['architecture_F']: 'a',
    'control': 'd'
}

significance_letters_low_sensors = {
    new_names['architecture_A']: 'd',
    new_names['architecture_B']: 'c',
    new_names['architecture_C']: 'e',
    new_names['architecture_D']: 'd',
    new_names['architecture_E']: 'b',
    new_names['architecture_F']: 'a',
    'control': 'd'
}

# Function to determine plot properties based on the output file
def get_plot_properties(output_file):
    if 'crop%' in output_file.lower():
        return 'Absorbed PAR (%)', (70, 100)
    elif 'crop' in output_file.lower():
        return 'Absorbed PAR (umol m-2 s-1)', (200, 300)
    elif '_%' in output_file.lower():
        return 'Absorbed PAR (%)', (0, 30)
    else:
        return 'Absorbed PAR (umol m-2 s-1)', (600, 1500)

# Function to create box plots
def create_boxplot(data, y, output_file, sig_letters):
    plt.figure(figsize=(10, 6))
    architectures = sorted(data['architecture'].unique())  # Ensure consistent order
    data_to_plot = [data[data['architecture'] == arch][y] for arch in architectures]

    # Boxplot with labels
    plt.boxplot(data_to_plot, labels=architectures)
    plt.xticks(rotation=45, fontsize=10)

    # Add significance letters
    for i, architecture in enumerate(architectures):
        letter = sig_letters.get(architecture, '')
        arch_data = data[data['architecture'] == architecture]
        mean_value = arch_data[y].mean()
        std_dev = arch_data[y].std()
        y_position = mean_value + std_dev * 3 if pd.notnull(mean_value) else 0
        plt.text(x=i + 1, y=y_position, s=letter, ha='center', va='bottom', fontsize=12, color='black')

    # Set plot labels and limits
    ylabel, ylim = get_plot_properties(output_file)
    plt.ylabel(ylabel)
    plt.ylim(ylim)
    plt.xlabel('')
    plt.tight_layout()
    plt.savefig(output_file)
    plt.close()

# Generate plots for all datasets
plot_configs = [
    (data_high_totalPAR_plants, 'absorbedPAR_umol_m2_s1', 'plots/boxplot_total_absorbedPAR_high_density.png', significance_letters_high_plant),
    (data_low_totalPAR_plants, 'absorbedPAR_umol_m2_s1', 'plots/boxplot_total_absorbedPAR_low_density.png', significance_letters_low_plant),
    (data_high_crop, 'absorbedPAR_umol_m2_s1_crop', 'plots/boxplot_absorbedPAR_crop_high_density.png', significance_letters_high_sensors_crop),
    (data_low_crop, 'absorbedPAR_umol_m2_s1_crop', 'plots/boxplot_absorbedPAR_crop_low_density.png', significance_letters_low_sensors_crop),
    (data_high_crop, 'absorbedPAR_umol_m2_s1_crop%', 'plots/boxplot_absorbedPAR_crop%_high_density.png', significance_letters_high_sensors_crop),
    (data_low_crop, 'absorbedPAR_umol_m2_s1_crop%', 'plots/boxplot_absorbedPAR_crop%_low_density.png', significance_letters_low_sensors_crop),
    (data_high_crop, 'absorbedPAR_umol_m2_s1', 'plots/boxplot_absorbedPAR_umol_m2_s1_high_density.png', significance_letters_high_sensors),
    (data_low_crop, 'absorbedPAR_umol_m2_s1', 'plots/boxplot_absorbedPAR_umol_m2_s1_low_density.png', significance_letters_low_sensors),
    (data_high_crop, 'absorbedPAR_%', 'plots/boxplot_absorbedPAR_%_high_density.png', significance_letters_high_sensors),
    (data_low_crop, 'absorbedPAR_%', 'plots/boxplot_absorbedPAR_%_low_density.png', significance_letters_low_sensors)
]

for data, y_column, output_file, sig_letters in plot_configs:
    create_boxplot(data, y_column, output_file, sig_letters)

print("Bar plots and box plots generated successfully for all datasets.")


C:\Users\jacop\AppData\Local\Temp\ipykernel_1172\3085650955.py:101: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  plt.boxplot(data_to_plot, labels=architectures)
C:\Users\jacop\AppData\Local\Temp\ipykernel_1172\3085650955.py:101: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  plt.boxplot(data_to_plot, labels=architectures)
C:\Users\jacop\AppData\Local\Temp\ipykernel_1172\3085650955.py:101: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  plt.boxplot(data_to_plot, labels=architectures)
C:\Users\jacop\AppData\Local\Temp\ipykernel_1172\3085650955.py:101: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has b

Bar plots and box plots generated successfully for all datasets.


C:\Users\jacop\AppData\Local\Temp\ipykernel_1172\3085650955.py:101: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  plt.boxplot(data_to_plot, labels=architectures)


In [ ]:
'''PART 3: BAR PLOT DATA OF TOTAL ABS PAR FOR PLANTS AND SENSORS'''

import pandas as pd
import matplotlib.pyplot as plt

# Load datasets for high and low densities
data_high_totalPAR_plants = pd.read_csv('combined_files/combined_total_absorbedPAR_high_cleaned.csv')
data_low_totalPAR_plants = pd.read_csv('combined_files/combined_total_absorbedPAR_low_cleaned.csv')
data_high_crop = pd.read_csv('combined_files/combined_high_sensors_cleaned.csv')
data_low_crop = pd.read_csv('combined_files/combined_low_sensors_cleaned.csv')

# Function to calculate mean and std dev
def calculate_statistics(data):
    grouped = data.groupby(['architecture'])['absorbedPAR_umol_m2_s1']
    mean_values = grouped.mean().reset_index(name='mean')
    mean_values['std_dev'] = grouped.std().reset_index(name='std_dev')['std_dev']
    mean_values['n'] = grouped.count().reset_index(name='count')['count']
    return mean_values

# Calculate statistics for datasets
mean_values_high_totalPAR_plants = calculate_statistics(data_high_totalPAR_plants)
mean_values_low_totalPAR_plants = calculate_statistics(data_low_totalPAR_plants)
mean_values_high_crop = calculate_statistics(data_high_crop)
mean_values_low_crop = calculate_statistics(data_low_crop)


# Function to generate bar plots
def barplot_absorbedPAR(mean_values, density, output_file, significance_letters):
    fig, ax = plt.subplots(figsize=(10, 6))
    
    for architecture in mean_values['architecture'].unique():
        architecture_data = mean_values[mean_values['architecture'] == architecture]
        mean = architecture_data['mean'].iloc[0]
        std_dev = architecture_data['std_dev'].iloc[0]
        
        ax.bar(architecture, mean, yerr=std_dev, 
               width=0.5, edgecolor='black', linewidth=0.5, capsize=5, color='green')
        
        significance_letter = significance_letters.get(architecture, '')
        offset = std_dev * 0.1
        y_position = mean + std_dev + offset
        ax.text(architecture, y_position, s=significance_letter, ha='center', va='bottom', fontsize=12)

    y_label = 'Absorbed PAR per plant (umol m-2 s-1)' if 'crop' not in output_file.lower() else 'Absorbed PAR crop (umol m-2 s-1)'
    y_lim = (0, 1500) if 'crop' not in output_file.lower() else (0, 300)

    ax.set_ylabel(y_label)
    ax.set_ylim(y_lim)
    plt.xticks(rotation=45, fontsize=10)
    plt.tight_layout()
    plt.savefig(output_file)
    plt.close(fig)

# Dictionaries for significance letters
new_names = {        
    'architecture_A': 'A-Gradient45-90',
    'architecture_B': 'B-Gradient90-45',
    'architecture_C': 'C-Uniform90',
    'architecture_D': 'D-Uniform70',
    'architecture_E': 'E-Uniform50',
    'architecture_F': 'F-Uniform30'
}

significance_letters_high_plant = {
    new_names['architecture_A']: 'c',
    new_names['architecture_B']: 'b',
    new_names['architecture_C']: 'a',
    new_names['architecture_D']: 'b',
    new_names['architecture_E']: 'd',
    new_names['architecture_F']: 'e',
    'control': 'd'
}

significance_letters_low_plant = {
    new_names['architecture_A']: 'd',
    new_names['architecture_B']: 'b',
    new_names['architecture_C']: 'a',
    new_names['architecture_D']: 'ab',
    new_names['architecture_E']: 'e',
    new_names['architecture_F']: 'f',
    'control': 'c'
}

significance_letters_high_sensors_crop = {
    new_names['architecture_A']: 'b',
    new_names['architecture_B']: 'd',
    new_names['architecture_C']: 'a',
    new_names['architecture_D']: 'b',
    new_names['architecture_E']: 'e',
    new_names['architecture_F']: 'f',
    'control': 'c'
}

significance_letters_low_sensors_crop = {
    new_names['architecture_A']: 'b',
    new_names['architecture_B']: 'c',
    new_names['architecture_C']: 'a',
    new_names['architecture_D']: 'b',
    new_names['architecture_E']: 'd',
    new_names['architecture_F']: 'e',
    'control': 'b'
}

significance_letters_high_sensors = {
    new_names['architecture_A']: 'e',
    new_names['architecture_B']: 'c',
    new_names['architecture_C']: 'f',
    new_names['architecture_D']: 'e',
    new_names['architecture_E']: 'b',
    new_names['architecture_F']: 'a',
    'control': 'd'
}

significance_letters_low_sensors = {
    new_names['architecture_A']: 'd',
    new_names['architecture_B']: 'c',
    new_names['architecture_C']: 'e',
    new_names['architecture_D']: 'd',
    new_names['architecture_E']: 'b',
    new_names['architecture_F']: 'a',
    'control': 'd'
}


# Generate bar plots
barplot_absorbedPAR(mean_values_high_totalPAR_plants, 'High', 'plots/total_absorbedPAR_high_density.png', significance_letters_high_plant)
barplot_absorbedPAR(mean_values_low_totalPAR_plants, 'Low', 'plots/total_absorbedPAR_low_density.png', significance_letters_low_plant)
barplot_absorbedPAR(mean_values_high_crop, 'High', 'plots/crop_high_density.png', significance_letters_high_sensors)
barplot_absorbedPAR(mean_values_low_crop, 'Low', 'plots/crop_low_density.png', significance_letters_low_sensors)


In [ ]:
'''PART 4: LINE PLOT FOR RANKS of ABSORBED PAR OVER EACH LEAF RANK.'''

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import rankdata

# Load datasets for high and low densities
data_high = pd.read_csv('combined_files/combined_high_ranks_cleaned.csv')
data_low = pd.read_csv('combined_files/combined_low_ranks_cleaned.csv')

# Function to calculate ranks for absorbed PAR within each architecture and leaf rank
def calculate_ranks(data):
    # Group by architecture and rank, then calculate the rank for absorbed PAR
    data['rank_absorbedPAR'] = data.groupby(['architecture', 'rank'])['absorbedPAR_umol_m2_s1'].transform(lambda x: rankdata(x, method='average'))
    # Calculate the mean rank for each architecture and leaf rank
    mean_ranks = data.groupby(['architecture', 'rank'])['rank_absorbedPAR'].mean().reset_index(name='mean_rank')
    
    return mean_ranks

# Calculate ranks for both high-density and low-density datasets
mean_ranks_high = calculate_ranks(data_high)
mean_ranks_low = calculate_ranks(data_low)

# Plotting function to avoid repetition
def plot_ranked_absorbedPAR(mean_ranks, density_label, output_file):
    plt.figure(figsize=(10, 6))
    
    # Plotting lineplot for each architecture type within the current density
    for architecture in mean_ranks['architecture'].unique():
        architecture_data = mean_ranks[mean_ranks['architecture'] == architecture]
        sns.lineplot(x='rank', y='mean_rank', data=architecture_data, label=architecture, marker='o')
    
    #plt.title(f'Ranked Absorbed PAR over Leaf Ranks for {density_label} Density')
    plt.xlabel('Leaf Rank')
    plt.ylabel('Mean Ranked data of Absorbed PAR (umol m-2 s-1)')
    
    # Automatically adjust the y-axis based on data range
    plt.ylim(mean_ranks['mean_rank'].min(), mean_ranks['mean_rank'].max())
    plt.xticks(range(1, 11))
    plt.legend(title='Architecture Type')
    plt.savefig(output_file)
    plt.close()

# Plot for high-density data
plot_ranked_absorbedPAR(mean_ranks_high, 'High', 'plots/ranked_absorbedPAR_for_high_density.png')

# Plot for low-density data
plot_ranked_absorbedPAR(mean_ranks_low, 'Low', 'plots/ranked_absorbedPAR_for_low_density.png')




In [ ]:
'''PART 5: BAR PLOT OF RANK DATA OF TOTAL ABS PAR FOR PLANTS AND SENSORS'''

import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import rankdata

# Load datasets for high and low densities
data_high_totalPAR_plants = pd.read_csv('combined_files/combined_total_absorbedPAR_high_cleaned.csv')
data_low_totalPAR_plants = pd.read_csv('combined_files/combined_total_absorbedPAR_low_cleaned.csv')
data_high_sensors = pd.read_csv('combined_files/combined_high_sensors_cleaned.csv')
data_low_sensors = pd.read_csv('combined_files/combined_low_sensors_cleaned.csv')

# Function to calculate ranks
def calculate_ranks(data):
    # Rank the absorbed PAR values
    data['rank'] = rankdata(data['absorbedPAR_umol_m2_s1'])
    return data

# Apply rank calculation for both high-density and low-density datasets
data_high_totalPAR_plants = calculate_ranks(data_high_totalPAR_plants)
data_low_totalPAR_plants = calculate_ranks(data_low_totalPAR_plants)
data_high_sensors = calculate_ranks(data_high_sensors)
data_low_sensors = calculate_ranks(data_low_sensors)

# Function to calculate mean rank and standard deviation for each architecture
def calculate_statistics(data):
    grouped = data.groupby(['architecture'])['rank']
    mean_values = grouped.mean().reset_index(name='mean_rank')
    mean_values['std_dev'] = grouped.std().reset_index(name='std_dev')['std_dev']
    mean_values['n'] = grouped.count().reset_index(name='count')['count']
        
    return mean_values

# Calculate statistics for both high-density and low-density datasets
mean_values_high_totalPAR_plants = calculate_statistics(data_high_totalPAR_plants)
mean_values_low_totalPAR_plants = calculate_statistics(data_low_totalPAR_plants)
mean_values_high_sensors = calculate_statistics(data_high_sensors)
mean_values_low_sensors = calculate_statistics(data_low_sensors)

def barplot_ranked_absorbedPAR(mean_values, density_label, output_file, significance_letters):
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Plotting bar plot for each architecture type within the current density
    for architecture in mean_values['architecture'].unique():
        architecture_data = mean_values[mean_values['architecture'] == architecture]
        mean_rank = architecture_data['mean_rank'].iloc[0]
        std_dev = architecture_data['std_dev'].iloc[0]
        
        # Plotting the bar
        ax.bar(architecture, mean_rank, yerr=std_dev, 
               width=0.5, edgecolor='black', linewidth=0.5, capsize=5, color='green')
        
        # Get the significance letter for the current architecture
        significance_letter = significance_letters.get(architecture, '')

        # Calculate the position for the significance letter
        offset = std_dev * 0.1  # 10% of the error bar length
        y_position = mean_rank + std_dev + offset
        
        # Adding the significance letter above the error bar
        ax.text(architecture, y_position, s=significance_letter, 
                ha='center', va='bottom', fontsize=12)


    # Set the title and labels
    #ax.set_title(f'Total Ranked Absorbed PAR for {density_label} density')

    # Automatically determine the y-axis label based on the output file name
    if 'sensors' in output_file.lower():
        y_label = 'Ranked absorbed PAR sensors (umol m-2 s-1)'
    else:
        y_label = 'Ranked absorbed PAR per plant (umol m-2 s-1)'

    ax.set_ylabel(y_label)
    plt.xticks(rotation=45, fontsize=10)

    # Save the plot
    plt.tight_layout()
    plt.savefig(output_file)
    plt.close(fig)

# Example dictionaries containing significance letters for architectures
new_names = {        
    'architecture_A': 'A-Gradient45-90',
    'architecture_B': 'B-Gradient90-45',
    'architecture_C': 'C-Uniform90',
    'architecture_D': 'D-Uniform70',
    'architecture_E': 'E-Uniform50',
    'architecture_F': 'F-Uniform30'
}

significance_letters_high_plant_ranked = {
    new_names['architecture_A']: 'a',
    new_names['architecture_B']: 'b',
    new_names['architecture_C']: 'b',
    new_names['architecture_D']: 'b',
    new_names['architecture_E']: 'a',
    new_names['architecture_F']: 'c',
    'control': 'a'
}

significance_letters_low_plant_ranked = {
    new_names['architecture_A']: 'ab',
    new_names['architecture_B']: 'c',
    new_names['architecture_C']: 'c',
    new_names['architecture_D']: 'c',
    new_names['architecture_E']: 'a',
    new_names['architecture_F']: 'd',
    'control': 'b'
}

significance_letters_high_sensors_ranked = {
    new_names['architecture_A']: 'a',
    new_names['architecture_B']: 'b',
    new_names['architecture_C']: 'c',
    new_names['architecture_D']: 'ad',
    new_names['architecture_E']: 'e',
    new_names['architecture_F']: 'f',
    'control': 'bd'
}

significance_letters_low_sensors_ranked = {
    new_names['architecture_A']: 'a',
    new_names['architecture_B']: 'a',
    new_names['architecture_C']: 'b',
    new_names['architecture_D']: 'a',
    new_names['architecture_E']: 'c',
    new_names['architecture_F']: 'd',
    'control': 'a'
}

# Call the barplot function with significance letters
barplot_ranked_absorbedPAR(mean_values_high_totalPAR_plants, 'High', 'plots/ranked_total_absorbedPAR_high.png', significance_letters_high_plant_ranked)
barplot_ranked_absorbedPAR(mean_values_low_totalPAR_plants, 'Low', 'plots/ranked_total_absorbedPAR_low.png', significance_letters_low_plant_ranked)
barplot_ranked_absorbedPAR(mean_values_high_sensors, 'High', 'plots/ranked_sensors_high.png', significance_letters_high_sensors_ranked)
barplot_ranked_absorbedPAR(mean_values_low_sensors, 'Low', 'plots/ranked_sensors_low.png', significance_letters_low_sensors_ranked)

print("Ranked plots with significance letters generated successfully for high and low densities.")

